In [1]:
import tweets_processor
import numpy as np
import keras
import mlflow.keras
import talos as ta
import importlib
import os
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Embedding,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.backend import expand_dims
from sklearn.model_selection import train_test_split
from collections import Counter
from keras import regularizers
from keras.optimizers import Adam, Nadam, RMSprop
from keras.activations import softmax, relu, tanh
from keras.losses import categorical_crossentropy, logcosh
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
importlib.reload(tweets_processor)
# load the data
# get the tweets and the region labels from csv file
tweets_text, tweets_regions = tweets_processor.get_tweets_from_csv('../data/tweets_labelled_balanced.csv')

In [3]:
# preprocess the tweets
# processed_tweets = []
# for tweet in tweets_text:
#     processed_tweet = tweets_processor.preprocessor(tweet)
#     processed_tweets.append(processed_tweet)

In [3]:
# create the tokenizer at word level
#t = Tokenizer(lower = True, filters ='')
# create the tokenizer at character level
t = Tokenizer(lower = True, filters ='')
t.fit_on_texts(tweets_text)

In [4]:
# get the vocab size
vocab_size = len(t.word_counts) + 1

In [5]:
word_index = t.word_index

In [6]:
# convert the tweets to sequence of id's
encoded_tweets = t.texts_to_sequences(tweets_text)

# maximum input sequence length
# set it to max length tweet in the dataset
max_len = len(max(encoded_tweets, key=len))

# make inputs of same length by using pad_sequences
padded_tweets = pad_sequences(encoded_tweets,padding='post',maxlen=max_len)

In [7]:
# convert labels to categorical
categorical_labels = keras.utils.to_categorical(tweets_regions, num_classes=23)

In [8]:
# split the data into train and test
train_data, test_data, train_labels, test_labels = train_test_split(padded_tweets, categorical_labels, test_size=0.33, random_state=0)

In [9]:
# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open(os.path.join('../data', 'glove.twitter.27B.200d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [10]:
# prepare embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [12]:
# create model with keras
model = Sequential()

with mlflow.start_run():
    embedding_features = 200
    first_filter_num = 256
    first_conv_len = 4
    first_pool_size = 4
    second_filter_num = 256
    second_conv_len = 4
    second_pool_size = 4
    first_dense_dim = 256
    dropout = 0.2
    
    #create embedding for vocab size
    model.add(Embedding(vocab_size, output_dim=embedding_features, weights=[embedding_matrix], input_length=max_len, trainable=True)) # had to add input length for flattening later
    model.add(Conv1D(first_filter_num, first_conv_len, activation='relu'))
    model.add(Conv1D(first_filter_num, first_conv_len, activation='relu'))
    model.add(MaxPooling1D(first_pool_size))
    model.add(Dropout(dropout))
    model.add(Conv1D(second_filter_num, second_conv_len, activation='relu'))
    model.add(Conv1D(second_filter_num, second_conv_len, activation='relu'))
    model.add(MaxPooling1D(second_pool_size))
    model.add(Dropout(dropout))
    
    model.add(Flatten())
    model.add(Dense(first_dense_dim, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(len(tweets_processor.regions_mapping), activation='softmax')) # using our 23 regions
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    batch_size = 100
    epochs = 10
    callback = [keras.callbacks.EarlyStopping(monitor='loss'), ModelCheckpoint(filepath='cnn_classifier_twitter.h5', 
                             monitor='loss', save_best_only=True)]
    model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, callbacks = callback)
    
    score = model.evaluate(test_data, test_labels)
    print(score)
    
    #mlflow logs
    mlflow.log_param("max_input_len", max_len)
    mlflow.log_param("embedding_features", embedding_features)
    mlflow.log_param("first_filter_num", first_filter_num)
    mlflow.log_param("first_conv_len", first_conv_len)
    mlflow.log_param("first_pool_size", first_pool_size)
    mlflow.log_param("second_filter_num", second_filter_num)
    mlflow.log_param("second_conv_len", second_conv_len)
    mlflow.log_param("second_pool_size", second_pool_size)
    mlflow.log_param("first_dense_dim", first_dense_dim)
    mlflow.log_param("dropout", dropout)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_metric("evaluation_loss", score[0])
    mlflow.log_metric("evaluation_accuracy", score[1])
    mlflow.log_param("level", 'character_level')
    #mlflow.log_param("activation", activation) # default for now tanh
    
    mlflow.keras.log_model(model, "cnn_models")

Epoch 1/10
1175571/1175571 [==============================] - 1072s 912us/step - loss: 2.3494 - acc: 0.1807
Epoch 2/10
1175571/1175571 [==============================] - 1058s 900us/step - loss: 2.2084 - acc: 0.2369
Epoch 3/10
1175571/1175571 [==============================] - 1059s 901us/step - loss: 2.0308 - acc: 0.2927
Epoch 4/10
1175571/1175571 [==============================] - 1064s 905us/step - loss: 1.8632 - acc: 0.3501
Epoch 5/10
1175571/1175571 [==============================] - 1064s 905us/step - loss: 1.7249 - acc: 0.4024
Epoch 6/10
1175571/1175571 [==============================] - 1064s 906us/step - loss: 1.6007 - acc: 0.4498
Epoch 7/10
1175571/1175571 [==============================] - 1063s 905us/step - loss: 1.5003 - acc: 0.4869
Epoch 8/10
1175571/1175571 [==============================] - 1064s 905us/step - loss: 1.4252 - acc: 0.5119
Epoch 9/10
1175571/1175571 [==============================] - 1063s 905us/step - loss: 1.3654 - acc: 0.5318
Epoch 10/10
579013/579013 [=

In [51]:
predictions = model.predict(test_data)
label_preds = []
for pred in predictions:
    label_preds.append(np.argmax(pred))
Counter(label_preds)

Counter({14: 12930, 10: 149, 18: 121})

In [9]:
def cnn_model(x_train,y_train,x_val,y_val,params):
    # create model with keras
    model = Sequential()
    with mlflow.start_run():
        #create embedding for vocab size
        model.add(Embedding(vocab_size, output_dim=params['embedding_features'], input_length=max_len)) # had to add input length for flattening later
        model.add(Conv1D(params['first_filter_num'], params['first_conv_len'], activation=params['first_conv_activation']))
        model.add(Dropout(params['dropout']))
        model.add(Conv1D(params['first_filter_num'], params['first_conv_len'], activation=params['first_conv_activation']))
        model.add(MaxPooling1D(params['first_pool_size']))
        model.add(Conv1D(params['second_filter_num'], params['second_conv_len'], activation=params['second_conv_activation']))
        model.add(Dropout(params['dropout']))
        model.add(Conv1D(params['second_filter_num'], params['second_conv_len'], activation=params['second_conv_activation']))
        model.add(MaxPooling1D(params['second_pool_size']))
        #model.add(Dropout(dropout))

        model.add(Flatten())
        model.add(Dense(params['first_dense_dim'], activation='relu'))
        model.add(Dropout(params['dropout']))
        model.add(Dense(len(tweets_processor.regions_mapping), activation='softmax')) # using our 23 regions

        model.compile(loss=params['losses'],
                     optimizer=params['optimizer'],
                     metrics=['accuracy'])

        out = model.fit(train_data, train_labels, batch_size=params['batch_size'], epochs=params['epochs'], validation_data=[x_val, y_val])

        #mlflow logs
        mlflow.log_param("max_input_len", max_len)
        mlflow.log_param("embedding_features", params['embedding_features'])
        mlflow.log_param("first_filter_num", params['first_filter_num'])
        mlflow.log_param("first_conv_len", params['first_conv_len'])
        mlflow.log_param("first_pool_size", params['first_pool_size'])
        mlflow.log_param("second_filter_num", params['second_filter_num'])
        mlflow.log_param("second_conv_len", params['second_conv_len'])
        mlflow.log_param("second_pool_size", params['second_pool_size'])
        mlflow.log_param("first_dense_dim", params['first_dense_dim'])
        mlflow.log_param("dropout", params['dropout'])
        mlflow.log_param("batch_size", params['batch_size'])
        mlflow.log_param("epochs", params['epochs'])
        mlflow.log_param("model_loss", out.history['loss'])
        mlflow.log_param("model_accuracy", out.history['acc'])
        mlflow.log_param("model_validation_loss", out.history['val_loss'])
        mlflow.log_param("model_validation_accuracy", out.history['val_acc'])
        #mlflow.log_param("activation", activation) # default for now tanh

        #mlflow.keras.log_model(model, "cnn_models")
    
    return out, model

In [10]:
# hyperparameter tuning params
p = {'embedding_features': [50, 100, 200],
     'first_filter_num':[64, 128, 256],
     'first_conv_len':[3,4],
     'first_pool_size': [2,4],
     'second_filter_num': [64, 128, 256],
     'second_conv_len': [3,4],
     'second_pool_size':[2,4],
     'first_dense_dim':[128,256],
     'dropout':[0.2,0.5],
     'batch_size': [10,30,50,100],
     'epochs': [10,50,100],
     'optimizer': ['adam', 'rmsprop'],
     'losses': [categorical_crossentropy],
     'first_conv_activation':[relu, tanh],
     'second_conv_activation':[relu, tanh],
     'last_activation': [softmax]}

In [11]:
# talos scan
h = ta.Scan(train_data, train_labels,
          params=p,
          dataset_name='40k_tweets',
          experiment_no='2',
          model=cnn_model,
          grid_downsample=0.5)

  0%|          | 0/82944 [00:00<?, ?it/s]

Train on 26800 samples, validate on 8040 samples
Epoch 1/100
26800/26800 [==============================] - 40s 2ms/step - loss: 2.8775 - acc: 0.1266 - val_loss: 2.8219 - val_acc: 0.1296
Epoch 2/100
26800/26800 [==============================] - 39s 1ms/step - loss: 2.7745 - acc: 0.1476 - val_loss: 2.6937 - val_acc: 0.1552
Epoch 3/100
26800/26800 [==============================] - 42s 2ms/step - loss: 2.6412 - acc: 0.1825 - val_loss: 2.4705 - val_acc: 0.2163
Epoch 4/100
26800/26800 [==============================] - 40s 1ms/step - loss: 2.5013 - acc: 0.2205 - val_loss: 2.3032 - val_acc: 0.2720
Epoch 5/100
26800/26800 [==============================] - 40s 1ms/step - loss: 2.3491 - acc: 0.2605 - val_loss: 2.1712 - val_acc: 0.2969
Epoch 6/100
26800/26800 [==============================] - 40s 1ms/step - loss: 2.2106 - acc: 0.2954 - val_loss: 2.0738 - val_acc: 0.3271
Epoch 7/100
26800/26800 [==============================] - 41s 2ms/step - loss: 2.0904 - acc: 0.3257 - val_loss: 1.8333 - v

  0%|          | 1/82944 [1:05:49<91005:53:21, 3949.96s/it]

Train on 26800 samples, validate on 8040 samples
Epoch 1/50
26800/26800 [==============================] - 39s 1ms/step - loss: 2.9080 - acc: 0.1234 - val_loss: 2.8794 - val_acc: 0.1240
Epoch 2/50
26800/26800 [==============================] - 38s 1ms/step - loss: 2.8424 - acc: 0.1366 - val_loss: 2.8055 - val_acc: 0.1566
Epoch 3/50
26800/26800 [==============================] - 38s 1ms/step - loss: 2.7614 - acc: 0.1579 - val_loss: 2.6896 - val_acc: 0.1837
Epoch 4/50
26800/26800 [==============================] - 37s 1ms/step - loss: 2.6657 - acc: 0.1841 - val_loss: 2.5321 - val_acc: 0.2280
Epoch 5/50
26800/26800 [==============================] - 37s 1ms/step - loss: 2.5624 - acc: 0.2108 - val_loss: 2.3951 - val_acc: 0.2701
Epoch 6/50
26800/26800 [==============================] - 38s 1ms/step - loss: 2.4599 - acc: 0.2322 - val_loss: 2.3283 - val_acc: 0.2741
Epoch 7/50
26800/26800 [==============================] - 37s 1ms/step - loss: 2.3629 - acc: 0.2604 - val_loss: 2.1706 - val_acc:

  0%|          | 2/82944 [1:36:57<76613:42:51, 3325.33s/it]

...

  0%|          | 18/82944 [25:20:10<214968:25:30, 9332.25s/it]

Train on 26800 samples, validate on 8040 samples
Epoch 1/50
26800/26800 [==============================] - 43s 2ms/step - loss: 2.8956 - acc: 0.1249 - val_loss: 2.8298 - val_acc: 0.1313
Epoch 2/50
26800/26800 [==============================] - 40s 2ms/step - loss: 2.7761 - acc: 0.1457 - val_loss: 2.5915 - val_acc: 0.1933
Epoch 3/50
26800/26800 [==============================] - 40s 2ms/step - loss: 2.5454 - acc: 0.2032 - val_loss: 2.2816 - val_acc: 0.2879
Epoch 4/50
26800/26800 [==============================] - 40s 2ms/step - loss: 2.2692 - acc: 0.2788 - val_loss: 1.9998 - val_acc: 0.3501
Epoch 5/50
26800/26800 [==============================] - 40s 2ms/step - loss: 1.9864 - acc: 0.3494 - val_loss: 1.6945 - val_acc: 0.4445
Epoch 6/50
26800/26800 [==============================] - 40s 2ms/step - loss: 1.7387 - acc: 0.4257 - val_loss: 1.4593 - val_acc: 0.5150
Epoch 7/50
26800/26800 [==============================] - 40s 2ms/step - loss: 1.5259 - acc: 0.4884 - val_loss: 1.2437 - val_acc:

  0%|          | 19/82944 [25:54:06<164539:59:54, 7143.13s/it]

Train on 26800 samples, validate on 8040 samples
Epoch 1/50
26800/26800 [==============================] - 235s 9ms/step - loss: 2.9047 - acc: 0.1249 - val_loss: 2.8675 - val_acc: 0.1239
Epoch 2/50
26800/26800 [==============================] - 235s 9ms/step - loss: 2.8806 - acc: 0.1283 - val_loss: 2.9001 - val_acc: 0.1239
Epoch 3/50
26800/26800 [==============================] - 235s 9ms/step - loss: 2.8816 - acc: 0.1284 - val_loss: 2.8881 - val_acc: 0.1340
Epoch 4/50
 9770/26800 [=========>....................] - ETA: 2:26 - loss: 2.8903 - acc: 0.1322

KeyboardInterrupt: 